### 벡터 DB 에 샘플 데이터 저장

In [7]:
from openai import OpenAI
import sys
import psycopg2  
from dotenv import load_dotenv
import os


load_dotenv()  
STUDENT_NO = '47'
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

# PostgresSQL 접속정보

DB_CONFIG = {
  'host': os.getenv("HOST"),
  'port': 5432,
  'database': 'postgres',
  'user' : 'postgres',
  'password' : os.getenv("PASSWORD")
}

# 임베딩 함수
def get_embedding(text: str, model: str = "text-embedding-3-small") -> list:
    response = client.embeddings.create(
        input=text,
        model=model
    )
    print(response.data[0].embedding[0])
    return response.data[0].embedding

# DB에 저장
def save_to_postgres(text: str, vector: list):   
    conn = psycopg2.connect(**DB_CONFIG) 
    cur = conn.cursor()
    try:
        sql = """
          INSERT INTO test_vector (student_no, name, embedding)
          VALUES (%s, %s, %s)
        """
        cur.execute(sql, (STUDENT_NO, text, vector))
        conn.commit()
        print("데이터 저장 완료!")
    except Exception as e:
        print("DB 오류:", e)
        conn.rollback()
    finally:
        cur.close()
        conn.close()

# 메인 실행
if __name__ == "__main__":
    input_text = input("임베딩할 문장을 입력하세요: ")
    embedding_vector = get_embedding(input_text)
    save_to_postgres(input_text, embedding_vector)
    # print("--------------")
    # print(len(embedding_vector))
    # print(embedding_vector)

-0.008849585428833961
데이터 저장 완료!


In [8]:
# python과 openAI를 이용한 벡터 검색

import openai
import psycopg2  #  **“Python + C + PostgreSQL version 2”**라는 의미라고 보시면 됩니다.
import os
from dotenv import load_dotenv
from pgvector.psycopg2 import register_vector


load_dotenv()  
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# PostgresSQL 접속정보

DB_CONFIG = {
  'host': os.getenv("HOST"),
  'port': 5432,
  'database': 'postgres',
  'user' : 'postgres',
  'password' : os.getenv("PASSWORD")
}

# 임베딩 함수
def get_embedding(text: str, model: str = "text-embedding-3-small") -> list:
    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding


# 가장 유사한 row 함수 검색
def find_most_similar(text: str) :
  #임베딩 벡터 생성
  vector = get_embedding(text)
  
  #DB 연결
  conn = psycopg2.connect(**DB_CONFIG) 
  register_vector(conn)
  cur = conn.cursor()
  try:
      sql = """
        SELECT name,embedding <-> %s::vector AS distance FROM test_vector 
        ORDER BY distance ASC LIMIT 1;
      """
      cur.execute(sql, (vector,)) ## 튜플로 넘겨야 함 (vector) 는 튜플이 아님
      result = cur.fetchone()
      
      if result:
        matched_text, similarity = result
        print(f"\n 가장 유사한 문장: {matched_text} ")
        print(f"거리 (낮을수록 유사): {similarity:.6f}")
      else :
        print("X 유사한 문장을 찾을 수 없습니다.")
      
  except Exception as e:
      print("DB 오류:", e)

  finally:
      cur.close()
      conn.close()
  
  
# 메인 실행
if __name__ == "__main__":
    user_input = input("유사한 문장을 찾을 문장을 입력하세요: ")
    find_most_similar(user_input)



 가장 유사한 문장: 할머니의 생신은 1955년 2월 30일 입니다. 
거리 (낮을수록 유사): 1.221483


In [9]:
import openai
import sys
import os
from dotenv import load_dotenv
from openai import OpenAI 

load_dotenv()  

GEMINI_API_URL = "https://generativelanguage.googleapis.com/v1beta/openai"
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
LLM_ID = "gemini-2.0-flash-lite"

client = OpenAI(
    base_url=GEMINI_API_URL,
    api_key=GEMINI_API_KEY
)

def ai_chat(messages: list):
    # openai api package를 통해 gemini api를 호출합니다.
    print(f"GEMINI API 호출, MODEL={LLM_ID}")
    response = client.chat.completions.create(
        model=LLM_ID,
        messages=messages,
    )
    return response

if __name__ == '__main__':
    messages = []
    if len(sys.argv) < 2:   
        print(f"Usage: {sys.argv[0]} '질문내용'")
        sys.exit()
    
    # 명령줄 인자 전체를 합쳐서 질문으로 사용
    question = ' '.join(sys.argv[1:])
    messages.append({'role': 'user', 'content': question})
    
    response = ai_chat(messages=messages)
    print(response.choices[0].message.content)

GEMINI API 호출, MODEL=gemini-2.0-flash-lite
This looks like a file path to a Jupyter kernel configuration file.  Let's break it down:

*   **`--f=`**: This suggests an option or flag being passed to a command or program. The `f` likely stands for "file." It indicates that the following string is the path to a file.

*   **`/Users/hyunjunson/Library/Jupyter/runtime/`**: This is the directory path to where the Jupyter runtime files are stored.

*   **`kernel-v3a44e24bfca538dc6313b023393765685b93e85ae.json`**: This is the specific filename.

    *   **`kernel-`**: This likely indicates it's a kernel configuration file.
    *   **`v3a44e24bfca538dc6313b023393765685b93e85ae`**: This is a unique identifier (likely a UUID or similar) for this specific kernel instance.
    *   **.json**:  The file extension. It means the file is in JSON (JavaScript Object Notation) format, which is a common data format. Jupyter kernels use JSON files to store the configuration information needed to run a kernel

In [10]:
import sys
print(sys.executable)

/Users/hyunjunson/Project/green-hat/backend/ml-pipline/yes/envs/langCh-env/bin/python


In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/')
def hello() :
    return render_template('hello.html')

@app.route('/greeting')
def hello2() :
    return render_template('greeting.html')
  
if __name__ == "__main__" :
  app.run(host="0.0.0.0", port=5013)
  


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5013
 * Running on http://192.168.0.99:5013
Press CTRL+C to quit


In [ ]:
import os
import sys
from dotenv import load_dotenv
import psycopg2
from flask import Flask, render_template, request, g

app = Flask(__name__)

# 환경변수 로드
load_dotenv()

# DB 연결정보
db_config = {
    "host": os.getenv("DB_HOST"),
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASS"),
    "port": os.getenv("DB_PORT")
}

def get_db():
    if 'db' not in g:
        g.db = psycopg2.connect(**db_config)
    return g.db

@app.teardown_appcontext
def close_db(error):
    db = g.pop('db', None)
    if db is not None:
        db.close()

# main page route
@app.route("/")
def index():
    # test_vector 테이블의 데이터를 페이지네이션 해서 표시
    page = request.args.get('page', 1, type=int)
    per_page = 10

    try:
        conn = get_db()
        cursor = conn.cursor()

        # 전체 항목 수 계산
        cursor.execute("SELECT COUNT(*) FROM test_vector;")
        total_items = cursor.fetchone()[0]
        total_pages = (total_items + per_page - 1) // per_page

        # 현재 페이지 데이터 조회 (offset, limit)
        offset = (page - 1) * per_page
        query = """
            SELECT id, student_no, name
            FROM test_vector
            ORDER BY id DESC
            LIMIT %s OFFSET %s;
        """
        cursor.execute(query, (per_page, offset))
        documents = cursor.fetchall()

    except psycopg2.Error as e:
        print(f"Database error: {e}")
        documents = []
        total_pages = 0

    finally:
        if 'cursor' in locals():
            cursor.close()

    return render_template(
        "index.html",
        documents=documents,
        page=page,
        total_pages=total_pages
    )

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5013)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5013
 * Running on http://192.168.0.99:5013
Press CTRL+C to quit
127.0.0.1 - - [30/Sep/2025 16:26:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2025 16:26:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Sep/2025 16:26:42] "GET /?page=2 HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2025 16:26:44] "GET /?page=3 HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2025 16:26:46] "GET /?page=2 HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2025 16:26:47] "GET /?page=1 HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2025 16:33:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2025 16:33:52] "GET /?page=2 HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2025 16:33:53] "GET /?page=3 HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2025 16:33:55] "GET /?page=2 HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2025 16:33:56] "GET /?page=1 HTTP/1.1" 200 -
